In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [38]:
device = "cpu"

In [39]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [40]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [41]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

# opt = torch_numopt.ConjugateGradient(params=model.parameters(), model=model, lr=1e-4, line_search_method="const", cg_method="PR")
opt = torch_numopt.ConjugateGradient(params=model.parameters(), model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(params=model.parameters(), model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(params=model.parameters(), model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(params=model.parameters(), model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein", cg_method="PR")


all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.1955823302268982
epoch:  1, loss: 0.12823089957237244
epoch:  2, loss: 0.08811270445585251
epoch:  3, loss: 0.06461028009653091
epoch:  4, loss: 0.05101602524518967
epoch:  5, loss: 0.04324067011475563
epoch:  6, loss: 0.03883751481771469
epoch:  7, loss: 0.03636486828327179
epoch:  8, loss: 0.034985460340976715
epoch:  9, loss: 0.03421953693032265
epoch:  10, loss: 0.03379528969526291
epoch:  11, loss: 0.03356044739484787
epoch:  12, loss: 0.03343008831143379
epoch:  13, loss: 0.03335714712738991
epoch:  14, loss: 0.03331570327281952
epoch:  15, loss: 0.03329147771000862
epoch:  16, loss: 0.033276621252298355
epoch:  17, loss: 0.033261869102716446
epoch:  18, loss: 0.03324167802929878
epoch:  19, loss: 0.03323246166110039
epoch:  20, loss: 0.03320414572954178
epoch:  21, loss: 0.03319245204329491
epoch:  22, loss: 0.03315110132098198
epoch:  23, loss: 0.0331377238035202
epoch:  24, loss: 0.03307965397834778
epoch:  25, loss: 0.033044055104255676
epoch:  26, loss: 0.

In [42]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.7150366249418955
Test metrics:  R2 = 0.6745394780094169


In [43]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

# opt = torch_numopt.ConjugateGradient(params=model.parameters(), model=model, lr=1e-4, line_search_method="const", cg_method="FR")
opt = torch_numopt.ConjugateGradient(params=model.parameters(), model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(params=model.parameters(), model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(params=model.parameters(), model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(params=model.parameters(), model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein", cg_method="FR")

all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.1170051097869873
epoch:  1, loss: 0.07843513786792755
epoch:  2, loss: 0.05806645378470421
epoch:  3, loss: 0.047126688063144684
epoch:  4, loss: 0.04114944115281105
epoch:  5, loss: 0.0378275103867054
epoch:  6, loss: 0.035966720432043076
epoch:  7, loss: 0.034918833523988724
epoch:  8, loss: 0.03432825207710266
epoch:  9, loss: 0.033993154764175415
epoch:  10, loss: 0.033801399171352386
epoch:  11, loss: 0.033690307289361954
epoch:  12, loss: 0.0336250364780426
epoch:  13, loss: 0.03358563780784607
epoch:  14, loss: 0.03357809782028198
epoch:  15, loss: 0.03353321924805641
epoch:  16, loss: 0.03350593149662018
epoch:  17, loss: 0.033502787351608276
epoch:  18, loss: 0.03347083553671837
epoch:  19, loss: 0.033451322466135025
epoch:  20, loss: 0.03344447910785675
epoch:  21, loss: 0.03342122212052345
epoch:  22, loss: 0.03340663015842438
epoch:  23, loss: 0.03339357301592827
epoch:  24, loss: 0.033376652747392654
epoch:  25, loss: 0.03337284177541733
epoch:  26, loss

In [44]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.8883654709902937
Test metrics:  R2 = 0.8763749073839097
